In [1]:
import sys, os
from numpy import *
from matplotlib.pyplot import *
%matplotlib inline
matplotlib.rcParams['savefig.dpi'] = 100

%load_ext autoreload
%autoreload 2

A baseline softmax classifier

Data preprocessing

In [2]:
import data_utils.utils as du
import data_utils.ner_base as nerbase
import data_utils.ner as ner

In [3]:
# # DONE: generating training set
# import json

# f = open('data/train','w')

# with open('data/buzs_labeled.json') as json_file:
#     for line in json_file:
#         review = json.loads(line)
#         for tuple in review["text"]:
#             tuple = tuple[0] + "\t" + tuple[1] + "\n"
#             data = tuple.encode("ascii","ignore")
#             if data.strip() == "-":
#                 data = "\n"
#             f.write(data)
            

In [4]:
# VOCAB_SIZE = 10000

# # TODO: add food word

# # generating vocab
# fv = open('data/vocab.txt', 'w')
# fv.write("UUUNKKK\n")
# fv.write("<s>\n")
# fv.write("</s>\n")

# cnt = 0
# with open('data/vocabulary.txt') as vocab_file:
#     for line in vocab_file:
#         if cnt < VOCAB_SIZE:
#             word = line.split(' ')[0] + '\n'
#             if word.strip() != '':
#                 fv.write(word.encode("ascii","ignore"))
#         cnt += 1


In [5]:
# stop word and UUUNKKK should include in the dictionary
# Load the starter word vectors
wv, word_to_num, num_to_word = ner.load_wv('data/vocab.txt',
                                           'data/wordVectors.txt')

tagnames = ["O", "B", "I", "L", "U"]
num_to_tag = dict(enumerate(tagnames))
tag_to_num = du.invert_dict(num_to_tag)

wsize = 5
pad = (wsize - 1)/2

X_train, y_train, words_train = du.generateData('data/train_v2', word_to_num, tag_to_num, wsize)
X_dev, y_dev, words_dev = du.generateData('data/dev_v2', word_to_num, tag_to_num, wsize)


In [6]:
print words_train[0]
word_to_num['<s>']

['<s>' '<s>' 'Excellent' 'Chinese' 'food']


1

In [4]:
print [[num_to_word[x] for x in X_train[i]] for i in range(20)]
print [tagnames[y_train[i]] for i in range(20)]

[['<s>', '<s>', 'excellent', 'chinese', 'food'], ['<s>', 'excellent', 'chinese', 'food', ','], ['excellent', 'chinese', 'food', ',', 'although'], ['chinese', 'food', ',', 'although', 'there'], ['food', ',', 'although', 'there', 'was'], [',', 'although', 'there', 'was', 'too'], ['although', 'there', 'was', 'too', 'much'], ['there', 'was', 'too', 'much', 'broccoli'], ['was', 'too', 'much', 'broccoli', 'in'], ['too', 'much', 'broccoli', 'in', 'my'], ['much', 'broccoli', 'in', 'my', 'beef'], ['broccoli', 'in', 'my', 'beef', 'and'], ['in', 'my', 'beef', 'and', 'broccoli'], ['my', 'beef', 'and', 'broccoli', 'for'], ['beef', 'and', 'broccoli', 'for', 'my'], ['and', 'broccoli', 'for', 'my', 'taste'], ['broccoli', 'for', 'my', 'taste', '.'], ['for', 'my', 'taste', '.', '</s>'], ['my', 'taste', '.', '</s>', '</s>'], ['<s>', '<s>', 'the', 'food', 'is']]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'U', 'O', 'O', 'B', 'I', 'L', 'O', 'O', 'O', 'O', 'O']


# Keras

In [7]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Merge 
from keras.optimizers import SGD
from keras.layers.embeddings import Embedding
from keras.regularizers import l2


####! not same length ###
print len(X_train)
def one_hot(dim, x):
    res = zeros(dim)
    res[x] = 1.0
    return res
Y_train = [one_hot(5, x) for x in y_train]
Y_dev = [one_hot(5, x) for x in y_dev]

385851


Using gpu device 0: GeForce GT 750M


In [8]:
def wordvector_neighbors(idxs, wordVecs, num_to_word, n=10):
    res_list = []
    for idx in idxs:
        #print square(wordVecs - wordVecs[idx]).shape
        res = argsort(sum(square(wordVecs - wordVecs[idx]), axis=1))[:n+1]
        res_list.append([num_to_word[x] for x in res])
    return res_list

## model 1

In [35]:

windowSize = 5
middleDim = 300
ntrain = len(X_train)

model = Sequential()

embedding = Embedding(wv.shape[0], wv.shape[1], weights=[wv])
hidden = Dense(wv.shape[1] * windowSize, middleDim, W_regularizer = l2(.001))
output = Dense(middleDim, 5, W_regularizer = l2(.001))

model.add(embedding)
model.add(Flatten())
model.add(hidden)
model.add(Activation('tanh'))
model.add(output)
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adagrad')
%time model.fit(X_train[:ntrain], Y_train[:ntrain], nb_epoch=20, batch_size=128)

Epoch 0
Epoch 0
Epoch 0
385851/385851 [==============================] - 79s - loss: 0.2946    
Epoch 1
Epoch 1
Epoch 1
385851/385851 [==============================] - 76s - loss: 0.2525    
Epoch 2
Epoch 2
Epoch 2
385851/385851 [==============================] - 75s - loss: 0.2383    
Epoch 3
Epoch 3
Epoch 3
385851/385851 [==============================] - 72s - loss: 0.2291    
Epoch 4
Epoch 4
Epoch 4
385851/385851 [==============================] - 74s - loss: 0.2227    
Epoch 5
Epoch 5
Epoch 5
385851/385851 [==============================] - 74s - loss: 0.2176    
Epoch 6
Epoch 6
Epoch 6
385851/385851 [==============================] - 74s - loss: 0.2137    
Epoch 7
Epoch 7
Epoch 7
385851/385851 [==============================] - 73s - loss: 0.2102    
Epoch 8
Epoch 8
Epoch 8
385851/385851 [==============================] - 74s - loss: 0.2074    
Epoch 9
Epoch 9
Epoch 9
385851/385851 [==============================] - 74s - loss: 0.2050    
Epoch 10
Epoch 10
Epoch 10
385851/385851

{'accuracy': [],
 'epoch': 19,
 'epoch_size': [],
 'loss': [0.29455469539686163,
  0.54701436746435261,
  0.7853086649070159,
  1.0144396609847954,
  1.2371569258297939,
  1.4547914335226764,
  1.6684493525189594,
  1.8786952083658601,
  2.0860618742544093,
  2.2910191378175875,
  2.4938668459518758,
  2.6947639561444072,
  2.8940611242983172,
  3.0918469220123685,
  3.2880223534773347,
  3.4829138309708059,
  3.6765825060864454,
  3.8691151073745749,
  4.0605256469672382,
  4.2510891045405526],
 'val_accuracy': [],
 'val_loss': []}

In [38]:
model.save_weights('model1_params')

ImportError: No module named h5py

In [36]:
y_pred = model.predict_classes(X_train)
nerbase.full_report(y_train, y_pred, tagnames)
nerbase.eval_performance(y_train, y_pred, tagnames)

385851/385851 [==============================] - 7s     
             precision    recall  f1-score   support

          O     0.9556    0.9688    0.9621    331052
          B     0.7410    0.6867    0.7128     15705
          I     0.6776    0.5582    0.6122      5385
          L     0.7270    0.6900    0.7080     15705
          U     0.6960    0.6306    0.6617     18004

avg / total     0.9215    0.9245    0.9227    385851

=== Performance (omitting 'O' class) ===
Mean precision:  71.60%
Mean recall:     65.66%
Mean F1:         68.48%


In [34]:
y_pred = model.predict_classes(X_train)
nerbase.full_report(y_train, y_pred, tagnames)
nerbase.eval_performance(y_train, y_pred, tagnames)

385851/385851 [==============================] - 9s     
             precision    recall  f1-score   support

          O     0.9509    0.9685    0.9596    331052
          B     0.7303    0.6457    0.6854     15705
          I     0.6438    0.5071    0.5674      5385
          L     0.6852    0.7087    0.6968     15705
          U     0.7123    0.5664    0.6311     18004

avg / total     0.9157    0.9196    0.9170    385851

=== Performance (omitting 'O' class) ===
Mean precision:  70.30%
Mean recall:     62.41%
Mean F1:         65.92%


In [37]:
y_pred = model.predict_classes(X_dev)
nerbase.full_report(y_dev, y_pred, tagnames)
nerbase.eval_performance(y_dev, y_pred, tagnames)

48184/48184 [==============================] - 0s     
             precision    recall  f1-score   support

          O     0.9298    0.9655    0.9473     41047
          B     0.6756    0.5387    0.5994      2157
          I     0.6359    0.4163    0.5032       663
          L     0.6641    0.5535    0.6038      2157
          U     0.6099    0.4546    0.5210      2160

avg / total     0.8882    0.8975    0.8912     48184

=== Performance (omitting 'O' class) ===
Mean precision:  64.86%
Mean recall:     50.64%
Mean F1:         56.81%


In [10]:
y_pred = model.predict_classes(X_dev)
nerbase.full_report(y_dev, y_pred, tagnames)
nerbase.eval_performance(y_dev, y_pred, tagnames)

48184/48184 [==============================] - 0s     
             precision    recall  f1-score   support

          O     0.9278    0.9664    0.9467     41047
          B     0.6716    0.5443    0.6013      2157
          I     0.6054    0.3725    0.4613       663
          L     0.6468    0.5841    0.6139      2157
          U     0.6589    0.4051    0.5017      2160

avg / total     0.8873    0.8970    0.8897     48184

=== Performance (omitting 'O' class) ===
Mean precision:  65.41%
Mean recall:     49.82%
Mean F1:         56.20%


In [24]:
samples = du.sample_wrong_predicts(y_pred, y_train, X_train, word_to_num)
for sample in du.translate_samples(samples, num_to_word, tagnames):
    print "==============="
    print sample

[('<s>', 'O', 'O'), ('for', 'O', 'O'), ('the', 'O', 'O'), ('entree', 'O', 'O'), (',', 'O', 'O'), ('my', 'O', 'O'), ('wife', 'O', 'O'), ('ordered', 'U', 'O'), ('the', 'O', 'O'), ('``', 'U', 'O'), ('UUUNKKK', 'O', 'U'), ("''", 'O', 'O'), ('UUUNKKK', 'O', 'O'), ('$', 'O', 'O'), ('dd', 'O', 'O'), ('UUUNKKK', 'O', 'O'), (',', 'O', 'O'), ('which', 'O', 'O'), ('is', 'O', 'O'), ('curry', 'O', 'U'), ('lamb', 'O', 'O'), ('with', 'O', 'O'), ('fresh', 'O', 'O'), ('onions', 'U', 'O'), (',', 'O', 'O'), ('tomatoes', 'B', 'O'), (',', 'L', 'O'), ('garlic', 'O', 'O'), (',', 'O', 'O'), ('and', 'O', 'O'), ('ginger', 'O', 'O'), ('simmered', 'O', 'O'), ('thick', 'O', 'O'), ('sauce', 'B', 'O'), ('.', 'L', 'O'), ('</s>', 'O', 'O')]
[('<s>', 'O', 'O'), ('after', 'O', 'O'), ('the', 'O', 'O'), ('uni', 'O', 'O'), (',', 'O', 'O'), ('squid', 'O', 'U'), (',', 'O', 'O'), ('and', 'O', 'O'), ('quail', 'O', 'B'), ('egg', 'O', 'L'), ('sushi', 'O', 'O'), (',', 'O', 'O'), ('we', 'O', 'O'), ('had', 'B', 'O'), ('our', 'L', '

In [11]:
W = hidden.get_weights()[0].T
L = embedding.get_weights()[0]
b1 = hidden.get_weights()[1]
b2 = output.get_weights()[1]
U = output.get_weights()[0].T
def print_scores(scores, words):
    for i in reversed(range(len(scores))):
        print "[%d]: (%.03f) %s" % (i, scores[i], words[i])

In [14]:


#### YOUR CODE HERE ####
topscores = []
topwords = []
scores = W[:,100:150].dot(L.T)
sort_idx = scores.argsort(axis=1)
for i in xrange(scores.shape[0]):
    topscores.append([])
    topwords.append([])
    for j in sort_idx[i][-10:]:
        topscores[i].append(scores[i][j])
        topwords[i].append(num_to_word[j])

In [15]:
neurons = [46,54,204,122,102] # change this to your chosen neurons
for i in neurons:
    print "Neuron %d" % i
    print_scores(topscores[i], topwords[i])

Neuron 46
[9]: (0.252) yuh
[8]: (0.248) wolf
[7]: (0.247) gall
[6]: (0.246) folles
[5]: (0.243) rooster
[4]: (0.240) shoulder
[3]: (0.238) brith
[2]: (0.234) marrow
[1]: (0.233) fuh
[0]: (0.229) bleaching
Neuron 54
[9]: (0.456) carbohydrates
[8]: (0.449) variables
[7]: (0.428) integer
[6]: (0.408) average
[5]: (0.407) calories
[4]: (0.403) tothis
[3]: (0.396) statistically
[2]: (0.393) computers
[1]: (0.392) households
[0]: (0.390) arithmetic
Neuron 204
[9]: (0.193) crawfish
[8]: (0.142) kimchi
[7]: (0.139) falafel
[6]: (0.137) liters
[5]: (0.136) gallon
[4]: (0.135) medium
[3]: (0.130) solanum
[2]: (0.130) tempranillo
[1]: (0.129) arabica
[0]: (0.129) parf
Neuron 122
[9]: (0.471) </s>
[8]: (0.286) haitians
[7]: (0.270) residing
[6]: (0.260) hard-of-hearing
[5]: (0.243) apathetic
[4]: (0.240) filipinos
[3]: (0.239) mediterranean-type
[2]: (0.239) orleanian
[1]: (0.238) dominicans
[0]: (0.238) cubans
Neuron 102
[9]: (0.381) quaalude
[8]: (0.367) deez
[7]: (0.330) bratwursts
[6]: (0.328)

In [16]:
print W.shape, L.shape, b1.shape, b2.shape, U.shape

(300, 250) (69740, 50) (300,) (5,) (5, 300)


In [26]:
wv_trained = embedding.get_weights()[0]
print wordvector_neighbors([word_to_num['equations']], wv_trained, num_to_word, n=20)
print word_to_num['ordered']
#print model.predict_classes(array([[75], [2226]]))


[['equations', 'equation', 'linear', 'differential', 'variables', 'approximation', 'parameters', 'discrete', 'geometry', 'generalization', 'mathematical', 'algebraic', 'generalized', 'finite', 'approximations', 'equilibrium', 'probability', 'algorithms', 'constants', 'quantum', 'semantics']]
75


In [41]:
count = zeros(len(word_to_num))
for idx in xrange(len(X_train)):
    if y_train[idx] == 4:
        count[X_train[idx][0]] += 1
    
print sum(count)
print [(num_to_word[i], count[i]) for i in argsort(count)[::-1][:150]]

9831.0
[('<s>', 1322.0), ('the', 601.0), (',', 436.0), ('and', 414.0), ('had', 271.0), ('a', 239.0), ('with', 222.0), ('UUUNKKK', 193.0), ('ordered', 188.0), ('of', 185.0), ('for', 134.0), ('got', 107.0), ('to', 89.0), ('was', 89.0), ('i', 82.0), ('on', 70.0), ('order', 66.0), ('is', 61.0), ('but', 52.0), ('in', 49.0), ('try', 49.0), ('get', 48.0), ('their', 48.0), ('that', 48.0), ('like', 45.0), ('rice', 41.0), ('we', 40.0), ('cheese', 39.0), ('love', 38.0), ('pizza', 38.0), ('tried', 37.0), ('side', 37.0), ('my', 37.0), ('-', 34.0), ('have', 33.0), ('salad', 33.0), ('d', 33.0), ('as', 31.0), ('or', 31.0), ('good', 28.0), ('about', 28.0), (':', 28.0), ('chicken', 28.0), ('out', 27.0), ('shrimp', 27.0), ('...', 27.0), ('it', 27.0), ('$', 24.0), ('enjoyed', 24.0), ('m', 24.0), ("'s", 24.0), ('tuna', 23.0), ('coconut', 23.0), ('salmon', 22.0), ('they', 22.0), ('from', 21.0), ('bacon', 21.0), ('your', 20.0), ('fries', 20.0), ('eggs', 20.0), ('were', 20.0), ('not', 19.0), ("n't", 19.0), ('

In [17]:
from nerwindow import softmax
h = tanh(L.dot(W[:,100:150].T) + b1)
y_hat = softmax(h.dot(U.T) + b2)
sort_id = y_hat.argsort(axis=0)
topscores = [[] for _ in range(5)]
topwords = [[] for _ in range(5)]
for i in xrange(5):
    for j in sort_id.T[i][-20:]:
        topscores[i].append(y_hat[j][i])
        topwords[i].append(num_to_word[j])

for i in range(1,5):
    print "Output neuron %d: %s" % (i, num_to_tag[i])
    print_scores(topscores[i], topwords[i])
    print ""


Output neuron 1: B
[19]: (0.898) yucatan
[18]: (0.841) kavkaz
[17]: (0.828) udd
[16]: (0.823) banh
[15]: (0.816) trahn
[14]: (0.804) queso
[13]: (0.794) nem
[12]: (0.792) corazon
[11]: (0.790) veel
[10]: (0.785) evo
[9]: (0.782) buckeye
[8]: (0.775) loo
[7]: (0.772) mauricio
[6]: (0.766) louisiana
[5]: (0.765) mih
[4]: (0.757) az
[3]: (0.753) boh
[2]: (0.752) toppled
[1]: (0.748) mee
[0]: (0.734) shirted

Output neuron 2: I
[19]: (0.709) del
[18]: (0.675) highmark
[17]: (0.601) malgr
[16]: (0.562) wwf
[15]: (0.544) marrow
[14]: (0.543) kickstarter
[13]: (0.542) di
[12]: (0.539) glacier
[11]: (0.526) rbc
[10]: (0.517) dore
[9]: (0.505) pond
[8]: (0.501) templeton
[7]: (0.496) poppy
[6]: (0.493) blotch
[5]: (0.491) somme
[4]: (0.489) diamond
[3]: (0.488) director
[2]: (0.487) lloyd
[1]: (0.482) gall
[0]: (0.481) hamon

Output neuron 3: L
[19]: (0.892) shoulder
[18]: (0.884) quaalude
[17]: (0.874) supermarionation
[16]: (0.868) wolfpack
[15]: (0.844) myoo
[14]: (0.826) gem
[13]: (0.819) b

In [31]:
from nerwindow import softmax
h = tanh(L.dot(W[:, 50:100].T) + b1)
y_hat = softmax(h.dot(U.T) + b2)
sort_id = y_hat.argsort(axis=0)
topscores = [[] for _ in range(5)]
topwords = [[] for _ in range(5)]
for i in xrange(5):
    for j in sort_id.T[i][-30:]:
        topscores[i].append(y_hat[j][i])
        topwords[i].append(num_to_word[j])
print word_to_num['equations']

for i in range(1,5):
    print "Output neuron %d: %s" % (i, num_to_tag[i])
    print_scores(topscores[i], topwords[i])
    print ""

62490
Output neuron 1: B
[29]: (0.796) equations
[28]: (0.781) variables
[27]: (0.763) constants
[26]: (0.756) resend
[25]: (0.743) axioms
[24]: (0.713) dimensionless
[23]: (0.700) finite
[22]: (0.677) parameter
[21]: (0.674) quantum
[20]: (0.671) discriminant
[19]: (0.670) thermodynamic
[18]: (0.646) equilibrium
[17]: (0.646) equation
[16]: (0.638) arithmetic
[15]: (0.629) matrix
[14]: (0.623) algebraic
[13]: (0.615) mathematical
[12]: (0.613) equivalence
[11]: (0.612) infinite
[10]: (0.608) cognitive
[9]: (0.608) vectors
[8]: (0.607) probability
[7]: (0.602) discrete
[6]: (0.602) algorithms
[5]: (0.599) extenuating
[4]: (0.596) differential
[3]: (0.593) ;p
[2]: (0.592) relativity
[1]: (0.587) spatial
[0]: (0.584) tothis

Output neuron 2: I
[29]: (0.703) hah
[28]: (0.689) finnegans
[27]: (0.643) petrel
[26]: (0.641) muh
[25]: (0.637) dri
[24]: (0.632) cedars
[23]: (0.616) shinzo
[22]: (0.613) seki
[21]: (0.607) sheh
[20]: (0.607) typhoon
[19]: (0.599) banh
[18]: (0.585) sago
[17]: (0.

In [78]:
for i in reversed(xrange(len(sort_id))):
    if sort_id.T[1][i] == word_to_num['ordered']:
        print i
        break
    
print len(sort_id)

69384
69740


## model 2

In [119]:

def get_label_for_X(X, Y):
    X_label = zeros((len(X), windowSize/2 * 5))
    for i in xrange(len(X)):
        label = array([])
        for j in xrange(windowSize/2):
            if X[i][j] == word_to_num['<s>']:
                label = append(label, zeros(5)) 
            else:
                label = append(label, Y[i - windowSize/2 + j])
        X_label[i] = label
    return X_label
X_train_label = get_label_for_X(X_train, Y_train)


In [8]:
print Y_train[:15]
print X_train_label[:15]

[array([ 1.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.]), array([ 0.,  0.,  0.,  0.,  1.]), array([ 1.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.]), array([ 0.,  1.,  0.,  0.,  0.]), array([ 0.,  0.,  1.,  0.,  0.]), array([ 0.,  0.,  0.,  1.,  0.])]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0

In [124]:
windowSize = 5
middleDim = 300
ntrain = len(X_train)


left = Sequential()
left.add(Embedding(wv.shape[0], wv.shape[1], weights=[wv]))
left.add(Flatten())
left.add(Dense(wv.shape[1] * windowSize, middleDim, W_regularizer = l2(.001)))

right = Sequential()
right.add(Dense(windowSize/2 * 5, middleDim, W_regularizer = l2(0.001)))

model = Sequential()

model.add(Merge([left, right], mode='sum'))
model.add(Activation('tanh'))
model.add(Dense(middleDim, 5, W_regularizer = l2(.001)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adagrad')
%time model.fit([X_train[:ntrain], X_train_label[:ntrain]], Y_train[:ntrain], nb_epoch=10, batch_size=128)


'''
left = Sequential()
left.add(Embedding(wv.shape[0], wv.shape[1], weights=[wv]))
left.add(Flatten())
left.add(Dense(wv.shape[1] * windowSize, middleDim, W_regularizer = l2(.001)))
left.add(Activation('tanh'))

right = Sequential()
right.add(Dense(windowSize/2 * 5, 20, W_regularizer = l2(0.001)))
right.add(Activation('tanh'))

model = Sequential()

model.add(Merge([left, right], mode='concat'))
model.add(Dense(middleDim + 20, 5, W_regularizer = l2(.001)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adagrad')
%time model.fit([X_train[:ntrain], X_train_label[:ntrain]], Y_train[:ntrain], nb_epoch=10, batch_size=128)'''

MemoryError: ('Error allocating 13948000 bytes of device memory (unspecified launch failure).', "you might consider using 'theano.shared(..., borrow=True)'")

In [112]:
def sequential_predict(X):
    y_pred = []
    prev = zeros((len(X), 5))
    for i in xrange(len(X)):
        label = array([])
        for j in xrange(windowSize/2):
            if X[i][j] == '<s>':
                label = append(label, zeros(5))
            else:
                label = append(label, prev[i - windowSize/2 + j])
        pred = model.predict_classes([[X[i]], [label]], batch_size=1, verbose=0)
        y_pred.append(pred[0])
        prev[i][pred[0]] = 1.0
    return y_pred
        

In [26]:
y_pred = sequential_predict(X_train)
nerbase.full_report(y_train, y_pred, tagnames)
nerbase.eval_performance(y_train, y_pred, tagnames)

             precision    recall  f1-score   support

          O       0.96      0.99      0.97    340758
          B       0.83      0.64      0.72     12927
          I       0.79      0.50      0.61      4883
          L       0.81      0.62      0.70     12927
          U       0.70      0.54      0.61      9831

avg / total       0.94      0.94      0.94    381326

=== Performance (omitting 'O' class) ===
Mean precision:  78.80%
Mean recall:     59.32%
Mean F1:         67.63%


In [113]:
y_pred = sequential_predict(X_dev)
nerbase.full_report(y_dev, y_pred, tagnames)
nerbase.eval_performance(y_dev, y_pred, tagnames)

             precision    recall  f1-score   support

          O     0.9268    0.9680    0.9470     41047
          B     0.6709    0.5406    0.5987      2157
          I     0.6147    0.3922    0.4788       663
          L     0.6628    0.5341    0.5915      2157
          U     0.6469    0.4231    0.5116      2160

avg / total     0.8867    0.8971    0.8895     48184

=== Performance (omitting 'O' class) ===
Mean precision:  65.60%
Mean recall:     48.93%
Mean F1:         55.90%


In [118]:
y_pred = sequential_predict(X_dev)
nerbase.full_report(y_dev, y_pred, tagnames)
nerbase.eval_performance(y_dev, y_pred, tagnames)

             precision    recall  f1-score   support

          O     0.9267    0.9680    0.9469     41047
          B     0.6807    0.5387    0.6014      2157
          I     0.6080    0.3650    0.4562       663
          L     0.6614    0.5234    0.5844      2157
          U     0.6311    0.4380    0.5171      2160

avg / total     0.8862    0.8968    0.8892     48184

=== Performance (omitting 'O' class) ===
Mean precision:  65.31%
Mean recall:     48.75%
Mean F1:         55.73%


## model 3

In [106]:
X_train_cap = array([array([x[0].isupper() for x in X]) for X in words_train]).astype(int)
X_dev_cap = array([array([x[0].isupper() for x in X]) for X in words_dev]).astype(int)

In [107]:
print X_train_cap[0]
print words_train[0]
print X_dev_cap[0]
print words_dev[0]
print [x[0].isupper() for x in words_dev[0]]

[0 0 1 1 0]
['<s>' '<s>' 'Excellent' 'Chinese' 'food']
[0 0 0 1 1]
['<s>' '<s>' 'the' 'Lemon' 'Pepper']
[False, False, False, True, True]


In [108]:

windowSize = 5
middleDim = 300
ntrain = len(X_train)

left = Sequential()
left.add(Embedding(wv.shape[0], wv.shape[1], weights=[wv]))
left.add(Flatten())
left.add(Dense(wv.shape[1] * windowSize, middleDim, W_regularizer = l2(.001)))

right = Sequential()
right.add(Dense(windowSize, middleDim, W_regularizer = l2(0.001)))

model = Sequential()

model.add(Merge([left, right], mode='sum'))
model.add(Activation('tanh'))
model.add(Dense(middleDim, 5, W_regularizer = l2(.001)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adagrad')
%time model.fit([X_train[:ntrain], X_train_cap[:ntrain]], Y_train[:ntrain], nb_epoch=10, batch_size=128)

Epoch 0
Epoch 0
Epoch 0
Epoch 0
Epoch 0
Epoch 0
Epoch 0
Epoch 0
Epoch 0
Epoch 0
Epoch 0
Epoch 0
Epoch 0
Epoch 0
Epoch 0
Epoch 0
385851/385851 [==============================] - 57s - loss: 0.2939    
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
385851/385851 [==============================] - 59s - loss: 0.2518    
Epoch 2
Epoch 2
Epoch 2
Epoch 2
Epoch 2
Epoch 2
Epoch 2
Epoch 2
Epoch 2
Epoch 2
Epoch 2
Epoch 2
Epoch 2
Epoch 2
Epoch 2
Epoch 2
385851/385851 [==============================] - 58s - loss: 0.2376    
Epoch 3
Epoch 3
Epoch 3
Epoch 3
Epoch 3
Epoch 3
Epoch 3
Epoch 3
Epoch 3
Epoch 3
Epoch 3
Epoch 3
Epoch 3
Epoch 3
Epoch 3
Epoch 3
385851/385851 [==============================] - 62s - loss: 0.2289    
Epoch 4
Epoch 4
Epoch 4
Epoch 4
Epoch 4
Epoch 4
Epoch 4
Epoch 4
Epoch 4
Epoch 4
Epoch 4
Epoch 4
Epoch 4
Epoch 4
Epoch 4
Epoch 4
385851/385851 [==============================] - 61s - loss: 0.2226    


{'accuracy': [],
 'epoch': 9,
 'epoch_size': [],
 'loss': [0.29394582775457706,
  0.54570444892527992,
  0.78330673061229739,
  1.0122426433710299,
  1.2348388123407061,
  1.4523808757283934,
  1.666053215171077,
  1.8762421214010574,
  2.0835072846690208,
  2.2881705725199488],
 'val_accuracy': [],
 'val_loss': []}

In [109]:
y_pred = model.predict_classes([X_dev, X_dev_cap])
nerbase.full_report(y_dev, y_pred, tagnames)
nerbase.eval_performance(y_dev, y_pred, tagnames)

48184/48184 [==============================] - 0s     
             precision    recall  f1-score   support

          O     0.9267    0.9674    0.9466     41047
          B     0.6415    0.5990    0.6195      2157
          I     0.5873    0.4872    0.5326       663
          L     0.6739    0.5012    0.5748      2157
          U     0.6744    0.3644    0.4731      2160

avg / total     0.8867    0.8964    0.8884     48184

=== Performance (omitting 'O' class) ===
Mean precision:  65.62%
Mean recall:     48.80%
Mean F1:         55.36%


# WindowMLP

In [44]:
nepoch = 5
N = nepoch * len(y_train)
k = 5 # minibatch size
print wv.shape


random.seed(10) # do not change this!
#### YOUR CODE HERE ####
def epochSchedule(n):
    for i in xrange(n):
        yield i % len(y_train)
        
def randomSchedule(n):
    i = 0
    while i < n:
        i += 1
        yield random.randint(0, len(y_train))
        
def randomMinibatchSchedule(n, k):
    i = 0
    while i < n / k:
        i += 1
        yield [random.randint(0, len(y_train)) for _ in xrange(k)]
        
def annealingAlpha(n, alpha, tau):
    for i in xrange(n):
        yield alpha * tau / max(i, tau)   

In [58]:
from nerwindow import WindowMLP
clf = WindowMLP(wv, windowsize=5, dims=[None, 200, 5],
                reg=0.001, alpha=0.01)
n = 10 * len(X_train)
print n
k = 100
%time clf.train_sgd(X=X_train, y=y_train, idxiter=randomMinibatchSchedule(n, k), alphaiter=annealingAlpha(n / k, 0.01, n / k / 10))


952300
Begin SGD...
  Seen 0 in 0.00 s
  [0]: mean loss 3.06684
  [9523]: mean loss 0.165779
SGD complete: 9523 examples in 307.86 seconds.
CPU times: user 5min 5s, sys: 3.15 s, total: 5min 9s
Wall time: 5min 7s


[(0, 3.0668426776904845), (9523, 0.16577878826707726)]

In [59]:
y_pred = clf.predict(X_train)
nerbase.full_report(y_train, y_pred, tagnames)
nerbase.eval_performance(y_train, y_pred, tagnames)

             precision    recall  f1-score   support

          O       0.97      0.98      0.97     85505
          B       0.80      0.70      0.74      3043
          I       0.71      0.48      0.58       910
          L       0.73      0.74      0.74      3043
          U       0.76      0.57      0.65      2729

avg / total       0.95      0.95      0.95     95230

=== Performance (omitting 'O' class) ===
Mean precision:  75.88%
Mean recall:     65.72%
Mean F1:         70.11%


In [100]:
y_pred = clf.predict(X_dev)
nerbase.full_report(y_dev, y_pred, tagnames)
nerbase.eval_performance(y_dev, y_pred, tagnames)

             precision    recall  f1-score   support

          O       0.93      0.97      0.95      8559
          B       0.43      0.35      0.39       255
          I       0.38      0.15      0.21       135
          L       0.40      0.42      0.41       255
          U       0.34      0.09      0.14       328

avg / total       0.87      0.90      0.88      9532

=== Performance (omitting 'O' class) ===
Mean precision:  38.38%
Mean recall:     25.39%
Mean F1:         28.66%


# Baseline

Feature extractor

In [7]:
# baseline features:

# context
# capitalize word
# previous tokens...
##### FEATURES TO ADD #####
def isCapital(x):
    if len(x) > 0:
        return x[0].isupper()
    return false

def extractFeatures(X, words, windowsize, word_to_num):
    fv1 = contextFeature(X, windowsize, word_to_num)
    fv2 = capitalFeature(words)
    fv = column_stack((fv1,fv2))
    return fv2

def contextFeature(X, windowsize, word_to_num):
    m = len(word_to_num)
    onehot_x = identity(m)

    wdim = m * windowsize
    N = X.shape[0]
    fvec = onehot_x[X].reshape(N, wdim)
    return fvec

def capitalFeature(words):
    fvec = array([isCapital(ww) for ww in words]).astype(int)
    return fvec

Training classifier

In [13]:
# X_train, y_train = du.docs_to_windows(docs_train, word_to_num, tag_to_num)
def trainSubset(size, X_train, y_train, words_train):
    X_train_s = X_train[:size]
    y_train_s = y_train[:size]
    words_train_s = words_train[:size]
    return X_train_s, y_train_s, words_train_s


# TRAIN_SIZE = 500
# X_s, y_s, w_s = trainSubset(TRAIN_SIZE, X_train, y_train, words_train)
X_s = X_train
y_s = y_train
w_s = words_train


In [ ]:
fv = extractFeatures(X_s, w_s, wsize, word_to_num)

In [11]:
### just trying###
X = X_train
m = len(word_to_num)
onehot_x = identity(m)

wdim = m * wsize
N = X.shape[0]
fvec = onehot_x[X].reshape(N, wdim)

In [14]:
print fv.shape

(95230,)


Training using sklearn logistic regression package

In [32]:
from sklearn import linear_model
lr = linear_model.LogisticRegression(solver='lbfgs', multi_class='multinomial')
lr.fit(fv, y_s)
y_pred = lr.predict(fv)
print nerbase.full_report(y_s, y_pred, tagnames)

             precision    recall  f1-score   support

          O       0.97      1.00      0.99       430
          B       1.00      0.79      0.88        19
          I       1.00      0.85      0.92        13
          L       1.00      0.84      0.91        19
          U       1.00      0.79      0.88        19

avg / total       0.97      0.97      0.97       500

None


Evaluation metric

In [33]:
print len(X_dev)
print len(X_train)

8486
9082


In [35]:
print fv.shape
print fvdev.shape

(500, 29989)
(8486, 29989)


Testing using dummy dataset

In [36]:
fvdev = extractFeatures(X_dev, words_dev, windowsize, word_to_num)
yp = lr.predict(fvdev)

nerbase.full_report(y_dev, yp, tagnames) # full report, helpful diagnostics

             precision    recall  f1-score   support

          O       0.88      1.00      0.93      7392
          B       0.25      0.00      0.00       403
          I       0.00      0.00      0.00       196
          L       0.00      0.00      0.00       403
          U       0.09      0.05      0.07        92

avg / total       0.78      0.87      0.81      8486

